In [1]:
from pathlib import Path

data_path = Path("data/")
image_path = data_path / "events"
train_dir = image_path / "train_scaled"
test_dir = image_path / "test_scaled"

train_dir, test_dir
output_dir = 'output'

In [2]:

import torch
torch.cuda.empty_cache()
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [3]:
if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"
device = torch.device(dev)

In [4]:
import matplotlib.pyplot as plt
import torch

from torch import nn
from torchvision import transforms

from torchvision.models import EfficientNet_V2_S_Weights, efficientnet_v2_s

weights = EfficientNet_V2_S_Weights.DEFAULT
model = efficientnet_v2_s(weights=weights).to(device)

In [5]:
auto_transforms = weights.transforms()

In [6]:
from torchvision import datasets
train_data = datasets.ImageFolder(root=train_dir, # target folder of images
                                  transform=auto_transforms, # transforms to perform on data (images)
                                  target_transform=None) # transforms to perform on labels (if necessary)
test_data = datasets.ImageFolder(root=test_dir,
                                 transform=auto_transforms)


In [7]:

class_names = train_data.classes
class_names

['Combat',
 'DestroyedBuildings',
 'Fire',
 'Humanitarian Aid and rehabilitation',
 'Military vehicles and weapons']

In [8]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset=train_data, batch_size=16, shuffle=True)
test_dataloader = DataLoader(dataset=test_data, batch_size=16,shuffle=False)

In [9]:
# Get the length of class_names (one output unit for each class)
output_shape = len(class_names)
# Recreate the classifier layer and seed it to the target device

model.classifier = torch.nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=output_shape, bias=True),
).to(device)

In [10]:
from torch import nn
loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1) # this is also called "criterion"/"cost function" in some places
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9)

In [11]:
torch.cuda.set_device(0)

In [12]:
next(model.parameters()).is_cuda

True

In [13]:
for layer in model.features.parameters():
    layer.requires_grad = True

In [14]:
# Set the random seeds
try:
    from going_modular.going_modular import engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !move pytorch-deep-learning\going_modular .
    !rmdir /s /q pytorch-deep-learning
    from going_modular.going_modular import engine

# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Setup training and save the results
results = engine.train(model=model,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       optimizer=optimizer,
                       loss_fn=loss_fn,
                       epochs=15,
                       device=device)

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")


  0%|          | 0/15 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.4089 | train_acc: 0.5188 | test_loss: 2.4491 | test_acc: 0.8500
Epoch: 2 | train_loss: 0.7278 | train_acc: 0.9062 | test_loss: 0.5605 | test_acc: 0.9250
Epoch: 3 | train_loss: 0.5026 | train_acc: 0.9656 | test_loss: 0.5301 | test_acc: 0.9375
Epoch: 4 | train_loss: 0.4701 | train_acc: 0.9875 | test_loss: 0.5135 | test_acc: 0.9750
Epoch: 5 | train_loss: 0.4555 | train_acc: 0.9906 | test_loss: 0.5136 | test_acc: 0.9750
Epoch: 6 | train_loss: 0.4632 | train_acc: 0.9844 | test_loss: 0.5147 | test_acc: 0.9375
Epoch: 7 | train_loss: 0.4381 | train_acc: 0.9906 | test_loss: 0.4945 | test_acc: 0.9625
Epoch: 8 | train_loss: 0.4249 | train_acc: 1.0000 | test_loss: 0.4853 | test_acc: 0.9750
Epoch: 9 | train_loss: 0.4240 | train_acc: 0.9969 | test_loss: 0.4906 | test_acc: 0.9625
Epoch: 10 | train_loss: 0.4338 | train_acc: 0.9938 | test_loss: 0.4935 | test_acc: 0.9500
Epoch: 11 | train_loss: 0.4211 | train_acc: 1.0000 | test_loss: 0.4949 | test_acc: 0.9625
Epoch: 12 | train_l

In [15]:
torch.save(model.state_dict(), 'weights.tf')